# Morse grid imports

Here we look at applied examples to import data for use in Dr Peter Morse 3D visualtsation software _(in prep.)_. Data must be exported as $1800 \times 3600$ px png files. This might change in future. 

Generally, the code in this tutorial runs rather slow as there is a large amount of interpolation. Have patience. 

Folder `../local` is used to download data and export pngs. 

In [9]:
#import warnings
#warnings.filterwarnings('ignore')

#config_file = "../agrid/agrid.py"
#with open(config_file) as f:
#    code = compile(f.read(), config_file, 'exec')
#    exec(code, globals(), locals())

# ... or use you Python path or present working directory:
import sys, os
sys.path.append('..')

from agrid.grid import Grid
from agrid.acc import download

import numpy as np

import vtk
from vtk.util.numpy_support import vtk_to_numpy

In [12]:
from agrid.features.morse_2019 import export_morse_png

## SMEAN

Download the mantel model from from Becker and Boschi, The University of Texas at Austin. See original paper [Becker and_Boschi 2002](http://www-udc.ig.utexas.edu/external/becker/becker_and_boschi_2002.pdf) for details.

In [13]:
! mkdir -p ../../data/smean
! wget -nc http://www-udc.ig.utexas.edu/external/becker/ftp/smean_grd.tgz \
    -O ../../data/smean/smean_grd.tgz
! tar -xvzf ../../data/smean/smean_grd.tgz -C ../../data/


File ‘../../data/smean/smean_grd.tgz’ already there; not retrieving.
x smean/
x smean/dvs.8.grd
x smean/dvs.15.grd
x smean/dvs.29.grd
x smean/dvs.17.grd
x smean/dvs.28.grd
x smean/convert
x smean/dvs.27.grd
x smean/dvs.22.grd
x smean/dvs.20.grd
x smean/dvs.6.grd
x smean/dvs.11.grd
x smean/dvs.1.grd
x smean/dvs.13.grd
x smean/depths.dat
x smean/dvs.9.grd
x smean/dvs.24.grd
x smean/dvs.7.grd
x smean/dvs.14.grd
x smean/dvs.4.grd
x smean/dvs.23.grd
x smean/README
x smean/dvs.5.grd
x smean/dvs.19.grd
x smean/dvs.16.grd
x smean/dvs.25.grd
x smean/dvs.10.grd
x smean/dvs.21.grd
x smean/dvs.12.grd
x smean/dvs.18.grd
x smean/dvs.26.grd
x smean/dvs.2.grd
x smean/dvs.3.grd


In this example, the data is global and we define a grid with the right rsolution from start. However, as we will see later, we could work on regional data as well. The exported png will always have a global coverage with alpha covering the excluded areas. 

We read depth values for smean from a provided ascii file. However we also have a regular depth dimention, defined as regular 50km depths. 

In [14]:
#columns from left to right represent radius,depth,density,Vpv,Vph,Vsv,Vsh,eta,Q-mu,Q-kappa
km = 1000



smean_depths = km * np.loadtxt('../../data/smean/depths.dat')[::-1]
max_depth = np.max(smean_depths)

depths = km * np.array([*range(0, 1000, 20)] + [*range(1000, int(max_depth)//km, 50)])

prem_d, prem_vsv, prem_vsh = km * np.loadtxt('../../../data/models/PREM_1s.csv', 
                              delimiter = ',', 
                              usecols = [1, 5, 6],
                              unpack=True)

prem_vsv = prem_vsv[prem_d < max_depth]
prem_vsh = prem_vsh[prem_d < max_depth]
prem_d = prem_d[prem_d < max_depth]

print(depths)

[      0   20000   40000   60000   80000  100000  120000  140000  160000
  180000  200000  220000  240000  260000  280000  300000  320000  340000
  360000  380000  400000  420000  440000  460000  480000  500000  520000
  540000  560000  580000  600000  620000  640000  660000  680000  700000
  720000  740000  760000  780000  800000  820000  840000  860000  880000
  900000  920000  940000  960000  980000 1000000 1050000 1100000 1150000
 1200000 1250000 1300000 1350000 1400000 1450000 1500000 1550000 1600000
 1650000 1700000 1750000 1800000 1850000 1900000 1950000 2000000 2050000
 2100000 2150000 2200000 2250000 2300000 2350000 2400000 2450000 2500000
 2550000 2600000 2650000 2700000 2750000 2800000]


In [15]:
world = Grid(crs_tgt = 4326, 
             left= -180, up=90, down= -90, right=180.0, 
             res=(0.09995,0.0999), 
             depths = depths)

array = np.empty((world.ny, world.nx, world.ds.coords['Z'].size))
array[:] = np.nan
print(world.ds)

<xarray.Dataset>
Dimensions:  (RGB: 3, X: 3601, X_edge: 3602, Y: 1801, Y_edge: 1802, Z: 87, Z_edge: 88)
Coordinates:
  * X        (X) float32 -180.0 -179.9 -179.8 -179.7 ... 179.7 179.8 179.9 180.0
  * Y        (Y) float32 -90.0 -89.9 -89.8 -89.7 -89.6 ... 89.7 89.8 89.9 90.0
  * Z        (Z) float32 0.0 20000.0 40000.0 ... 2700000.0 2750000.0 2800000.0
  * X_edge   (X_edge) float32 -180.04997 -179.94998 ... 179.94998 180.04997
  * Y_edge   (Y_edge) float32 -90.04995 -89.94995 ... 89.94995 90.04995
  * Z_edge   (Z_edge) float64 -1e+04 1e+04 3e+04 ... 2.775e+06 2.825e+06
  * RGB      (RGB) <U1 'R' 'G' 'B'
    XV       (Y, X) float32 dask.array<shape=(1801, 3601), chunksize=(180, 360)>
    YV       (Y, X) float32 dask.array<shape=(1801, 3601), chunksize=(180, 360)>
    lat      (Y, X) float32 dask.array<shape=(1801, 3601), chunksize=(180, 360)>
    lon      (Y, X) float32 dask.array<shape=(1801, 3601), chunksize=(180, 360)>
Data variables:
    *empty*


In [16]:
world.ds['PREM_VSV'] = (('Z'), world.change_coord(prem_vsv, prem_d, 'Z') )

Now, we can read SMEAN files. set_center alows us to read 0-360 as -180 - 180 lon. read_grid uses default nearest neighbour interpolation of datapoints to fit the predefined Morse grid. 

In [17]:
smean_array = np.empty((world.ny, world.nx, smean_depths.size))

for i, a in enumerate(smean_depths):
    print(a//km, end=', ')
    index_name = smean_depths//km - i # Get right file name for each depth
    fname = '../../data/smean/dvs.%s.grd'%(i+1)
    if os.path.isfile(fname):
        smean_array[:,:,i] = world.read_grid(fname, xyz = ('lon','lat','z'), set_center = True)
        
#array /= 100    #"The grid file values are given are percentage v_S wave 
#                #speed variation related to PREM, with layer averages removed."
        
#world.ds['SMEAN'] = (('Y', 'X', 'Z'), array)

50.0, 150.0, 250.0, 350.0, 450.0, 550.0, 650.0, 750.0, 850.0, 950.0, 1050.0, 1150.0, 1250.0, 1350.0, 1450.0, 1550.0, 1650.0, 1750.0, 1850.0, 1950.0, 2050.0, 2150.0, 2250.0, 2350.0, 2450.0, 2550.0, 2650.0, 2750.0, 2850.0, 

Finally, we export the 3D grids as png files for each depth slice. We store a set of original depth slices in ''morse/smean/z_orig/' and also the interpolated depth slices in 'morse/smean/z_even/'. This might take some time. 

In [18]:
smean_min = np.nanmin(smean_array)
smean_max = np.nanmax(smean_array)

print(smean_min, smean_max)

-8.249747276306152 6.737351894378662


In [19]:
new_smean_array  = world.change_coord(smean_array, old=smean_depths, new='Z')

In [19]:
np.shape(new_smean_array)

(1800, 3600, 87)

In [20]:
world.ds['SMEAN'] = (('Y', 'X', 'Z'), new_smean_array)

In [21]:
import pickle
pickle.dump(world, open( "../local/world_size.p", "wb" ), protocol=4)
#world = pickle.load( open( "../local/world.p", "rb" ) )





OSError: [Errno 22] Invalid argument

In [23]:
! mkdir -p ../local/morse/smean

In [24]:
for i, z in enumerate(world.ds['Z'][::-1]):
    report = export_morse_png(world,
                        data = world.ds['SMEAN'].isel(Z=i).values, 
                        png_name = '../local/morse/smean/%04d_smean.png'%(int(z)//km), 
                        v_min = -10, v_max = 10, 
                        png_format = 'RGB',
                        set_geometry = False)
    print(report)
    with open("../local/morse/smean/log_smean.txt", "a") as log_file:
        log_file.write(report)
    

../agrid/features.py:202: RuntimeWarning: All-NaN slice encountered
  report += '%.2f \t  %.2f \t %.2f \t  %s \n' % (np.nanmin(data),
../agrid/features.py:206: RuntimeWarning: All-NaN slice encountered
  report += '%.2f \t  %.2f \t %.2f \t  %s \n \n' % (np.nanmax(data),


../local/morse/smean/2800_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/2800_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
nan 	  0.50 	 127.50 	  127 
nan 	  0.50 	 127.50 	  127 
 

../local/morse/smean/2750_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/2750_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
nan 	  0.50 	 127.50 	  127 
nan 	  0.50 	 127.50 	  127 
 

../local/morse/smean/2700_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/2700_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
nan 	  0.50 	 127.50 	  127 
nan 	  0.50 	 127.50 	  127 
 

../local/morse/smea

../local/morse/smean/1550_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/1550_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.72 	  0.41 	 105.58 	  105 
1.02 	  0.55 	 140.48 	  140 
 

../local/morse/smean/1500_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/1500_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.74 	  0.41 	 105.31 	  105 
1.03 	  0.55 	 140.61 	  140 
 

../local/morse/smean/1450_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/1450_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.76 	  0.41 	 105.01 	  105 
1.04 	  0.55 	 140.77 	  140 
 

../local/m

../local/morse/smean/0720_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/0720_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.39 	  0.43 	 109.73 	  109 
1.03 	  0.55 	 140.68 	  140 
 

../local/morse/smean/0700_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/0700_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.39 	  0.43 	 109.73 	  109 
1.07 	  0.55 	 141.14 	  141 
 

../local/morse/smean/0680_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/0680_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.37 	  0.43 	 110.00 	  110 
1.06 	  0.55 	 141.07 	  141 
 

../local/m

../local/morse/smean/0220_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/0220_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.44 	  0.43 	 109.19 	  109 
1.74 	  0.59 	 149.68 	  149 
 

../local/morse/smean/0200_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/0200_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.40 	  0.43 	 109.63 	  109 
1.71 	  0.59 	 149.26 	  149 
 

../local/morse/smean/0180_smean.pnggreyscale: False
alpha: False
planes: 3
bitdepth: 8
interlace: 0
size: (3600, 1800)
../local/morse/smean/0180_smean.png 
min v: -10 max v: 10 bit depth: 8
bands: (1800, 3600, 3) interpolation: nearest
data 	  norm 	  to png 	 png 
-1.38 	  0.43 	 109.91 	  109 
1.94 	  0.60 	 152.24 	  152 
 

../local/m

## GLAD M15

Now let's import the model from [Bozdağ et al (2016)](https://academic.oup.com/gji/article/207/3/1739/2404568). Unfortunately, it appears that the data is not availible from any open repo (?!), but I belive that the group is willing to assist if contacted. 

The SMEAN data was easy to import, but exporting the GLAD-M15 data is more compicated as I only get access to an unstructured grid, in a normalised space. 

In [ ]:
# load a vtk file to reader
reader = vtk.vtkXMLUnstructuredGridReader()
reader.SetFileName('../local/GLAD-M15/reg_1_dvsv.vtu')
reader.Update()

# Get the coordinates of nodes in the mesh
nodes_vtk_array= reader.GetOutput().GetPoints().GetData()

#The data field is the first scalar in the vtu file
data_vtk_array = reader.GetOutput().GetPointData().GetArray(0)

print(data_vtk_array, nodes_vtk_array)

We get two arrays. Note that sizes are equal (Number Of Tuples: 4712064 and Number Of Tuples: 4712064) and the coordinate are 3D (NumberOfComponents: 3). Read coordinates and data as numpy arrays and have a look: 

In [ ]:
nodes_numpy_array = vtk_to_numpy(nodes_vtk_array)
X, Y, Z = nodes_numpy_array[:,0] , nodes_numpy_array[:,1] , nodes_numpy_array[:,2]
V = vtk_to_numpy(data_vtk_array)

print(X[:10],Y[:10],Z[:10],V[:10], sep='\n\n')

The coordinates are stored in the range [-1..1]. We assume a spherical Earth and compute an array with every points distance from centre. Function `sphere_to_layer()` takes a slice from the sphere and save to a 2D array. delta_r should be as small as possible, for best vertical resolution, but if too few points are used the vertical resolution gets bad. 

We loop through the harmonics of the sphere and read all points within a spherical shell defined by the radius +/- delta_r

In [ ]:
def sphere_to_layer(d, R, V, LAT, LON, xxx, yyy, 
                    delta_r = 32*km, 
                    min_s = 5000,
                    r_earth =  (6357*km + 6378*km) / 2, 
                    interpolation = 'nearest'):
    
    A = np.zeros_like(xxx).astype('float')
    A[:] = np.nan
    upper = 1-(d-delta_r)/r_earth
    lower = 1-(d+delta_r)/r_earth
    S = (R > lower) & (R < upper) # Select points in spherical shell
    s_sum = np.count_nonzero(S) # Check how many points in shell

    if s_sum >= min_s:
        A = interpolate.griddata((LON[S], LAT[S]),
                V[S], (xxx, yyy), method = interpolation)
    print(d//1000, 'km \t N=', sum(S),u'\t \N{BLACK RIGHT-POINTING TRIANGLE}', np.nanmean(A))
    return np.flipud(A) #Because ulike lat, rows start from top. 

R = np.sqrt(X*X + Y*Y + Z*Z) # The distance from each point to centre of spherical Earth. 

LAT = world.shape3[0]/180 * (np.arccos( Z / R) * 180/np.pi) 
LON = world.shape3[1]/360 * (np.arctan2(Y, X) * 180/np.pi + 180)

xxx, yyy = np.meshgrid(range(0, world.shape3[1]), range(world.shape3[0], 0, -1)) 

glad = np.zeros(world.shape3) # self.shape3 is a tuple of the models dimensions

for i, d in enumerate(world.ds['Z'].values):
    glad[:,:,i] = sphere_to_layer(d, R, V, LAT, LON, xxx, yyy)

Unfortunately there are not enugh data points at 1350, 1450, 2150 and 2250 km depth. We'd need to look in a broad spherical shell to get datapoints: 

In [ ]:
missing_i, missing_d = [], []

for i in range(glad.shape[2]):
    if (np.count_nonzero(np.isnan(glad[:,:,i]) ) > 1000):
        missing_i.append(i)
        missing_d.append(world.ds['Z'].values[i])
      
for i,d in zip(missing_i, missing_d):
    print(i, end=' ')
    glad[:,:,i] = sphere_to_layer(d, R, V, LAT, LON, xxx, yyy, delta_r = 45*km)    

However, we can be more precise with the first layer. 

In [ ]:
glad[:,:,0] = sphere_to_layer(50*km, R, V, LAT, LON, xxx, yyy, delta_r = 14*km)   
glad[:,:,1] = sphere_to_layer(150*km, R, V, LAT, LON, xxx, yyy, delta_r = 20*km)   

We save the array to the object and have a look. 

In [ ]:
world.ds['GLAD'] = (('Y', 'X', 'Z'), glad)
glad_min = np.nanmin(world.ds['GLAD'].values)
glad_max = np.nanmax(world.ds['GLAD'].values)

print(glad_min, glad_max)
#print(smean_min, smean_max)

z_map = 0
world.map_grid(world.ds['GLAD'].isel(Z=z_map), cmap='magma_r', vmin=-0.06, vmax=0.06)
#world.map_grid(world.ds['SMEAN'].isel(Z=z_map), cmap='magma_r', vmin=-0.06, vmax=0.06)

In [ ]:
world.ds['DIFF'] = world.ds['GLAD']-world.ds['SMEAN']

diff_min = np.nanmin(world.ds['DIFF'].values)
diff_max = np.nanmax(world.ds['DIFF'].values)

print(diff_min, diff_max)

world.map_grid(world.ds['DIFF'].isel(Z=z_map), cmap='BrBG', vmin=-0.06, vmax=0.06)

In [ ]:
! mkdir -p ../local/glad/16_bit

In [ ]:
for i, z in enumerate(world.ds['Z']):
    report = export_morse_png(world, world.ds['GLAD'].isel(Z=i).values, '../local/glad/16_bit/%04d_%s_glad.png'%(int(z)//km, i+1), 
                          v_min = glad_min, v_max = glad_max, 
                              set_geometry = False, 
                             png_format = 'RGB', 
                             bit_depth = 16)
    with open("../local/morse/glad/log_16bit.txt", "a") as log_file:
        log_file.write(report)
    print(report)
    

Save the grid: 

In [ ]:
world.save(world.ds, '../local/smean_and_glad.nc')

We could also interpolate new depth values to make a regular 3D grid. 

In [ ]:
world.ds['Z_NEW'] = range(0,2850*km, 100*km)

world.ds['SMEAN_INTER'] = ( ('Y', 'X', 'Z'), 
                           world.change_coord(world.ds['SMEAN'], world.ds['Z'], world.ds['Z_NEW']) )

In [ ]:
world.ds

# AuSREM

Australian crustal and lithospheric seismic model.

Can be downloaded from http://rses.anu.edu.au/seismology/AuSREM/Downloads/ We assume a local file: 

In [ ]:
fname = '../local/ausrem_SV_100.txt'
data = world.read_ascii(fname)

Make a mask, as the 'nearest neigbur' interpolation tested, will extrapolate values to all cells.

In [ ]:
alpha_master = np.isfinite(data)

plt.imshow(alpha_master)
plt.show()

Here, we test a number of different formats for the output png file: 

In [ ]:
for _format in ['L', 'LA', 'RGB', 'RGBA']:
    for _bit_depth in [8, 16]:
        for _interpol in ['nearest', 'linear', 'cubic']:
            data = world.read_ascii(fname, interpol =_interpol)
            
            if _interpol == 'nearest':
                how_confine = 'mask'
                mask_to_value = np.nan
            else:
                how_confine = 'input'
                mask_to_value = None
            
            report = export_morse_png(world,
                          data = data, 
                          png_format = _format,
                          png_name = '../local/AuSREM/AuSREM_100km_%s_%sbit_%s.png'%( _interpol,_bit_depth,_format),
                          interpol_method =  _interpol,
                          confine_data = how_confine,
                          confine_mask = alpha_master,
                        mask_to_value = mask_to_value,
                          v_min = 4, v_max = 5, 
                          clip=True,
                          set_geometry = False, 
                          bit_depth=_bit_depth)

            print(report)
            with open("../local/AuSREM_log.txt", "a") as log_file:
                log_file.write(report)

In [ ]:
!sips -g all ../local/AuSREM/AuSREM_100km_nearest_16bit_L.png
#!open ../local/new/AuSREM_100km_nearest_16bit_L.png

In [25]:
world.ds

<xarray.Dataset>
Dimensions:   (RGB: 3, X: 3600, Y: 1800, Z: 87)
Coordinates:
  * X         (X) float32 -180.0 -179.9 -179.8 ... 179.70004 179.80005 179.90005
  * Y         (Y) float32 89.9001 89.8001 89.700096 ... -89.8 -89.9 -90.0
  * Z         (Z) float32 0.0 20000.0 40000.0 ... 2700000.0 2750000.0 2800000.0
  * RGB       (RGB) <U1 'R' 'G' 'B'
    XV        (Y, X) float32 dask.array<shape=(1800, 3600), chunksize=(180, 360)>
    YV        (Y, X) float32 dask.array<shape=(1800, 3600), chunksize=(180, 360)>
    lat       (Y, X) float32 dask.array<shape=(1800, 3600), chunksize=(180, 360)>
    lon       (Y, X) float32 dask.array<shape=(1800, 3600), chunksize=(180, 360)>
Data variables:
    PREM_VSV  (Z) float64 0.0 3.9e+03 4.4e+03 ... 7.247e+03 7.266e+03 7.266e+03
    SMEAN     (Y, X, Z) float64 nan nan nan nan nan ... 2.237 2.713 3.188 3.545

In [27]:
world.map_grid('SMEAN'.sel(Z=1000) )

SyntaxError: invalid syntax (<ipython-input-27-5895aaf965ad>, line 1)

In [ ]:
lat, lon, persent